In [1]:
import sys

In [2]:
sys.getrefcount?

Signature: sys.getrefcount(object, /)
Docstring:
Return the reference count of object.

The count returned is generally one higher than you might expect,
because it includes the (temporary) reference as an argument to
getrefcount().
Type:      builtin_function_or_method

In [3]:
sys.getrefcount([1, 2, 3])

1

In [4]:
lst = [1, 2, 3]
sys.getrefcount(lst)

2

In [5]:
lst2 = lst

In [6]:
sys.getrefcount(lst)

3

In [7]:
del lst

In [8]:
sys.getrefcount(lst2)

2

In [10]:
sys.getrefcount(5), sys.getrefcount(-5), sys.getrefcount(256), sys.getrefcount(0)

(3221225472, 3221225472, 3221225472, 3221225472)

In [11]:
sys.getrefcount(-6), sys.getrefcount(257), sys.getrefcount(2 ** 101)

(2, 3, 1)

In [12]:
st1 = "qwerty123"

In [13]:
st2 = "qwerty123"

In [14]:
st1 is st2

True

In [15]:
sys.getrefcount(st1)

3

In [16]:
st1 = "qwerty123!"

In [17]:
st2 = "qwerty123!"

In [18]:
st1 is st2

False

In [19]:
st1 = "буквы123"

In [20]:
st2 = "буквы123"

In [21]:
st1 is st2

False

In [22]:
буква = 12

In [23]:
буква

12

In [ ]:
sys.getrefcount(lst)

In [30]:
lst = [1, 2, 3, 4]

def calc_data(obj):
    print(obj is lst)
    print(f"calc_data {sys.getrefcount(obj)=}")
    print(f"calc_data {sys.getrefcount(lst)=}")


print(f"before {sys.getrefcount(lst)=}")

calc_data(lst)

print(f"after {sys.getrefcount(lst)=}")

before sys.getrefcount(lst)=2
True
calc_data sys.getrefcount(obj)=2
calc_data sys.getrefcount(lst)=3
after sys.getrefcount(lst)=2


In [31]:
class Animal:
    def __init__(self, name, food=None):
        self.name = name
        self.food = food

    def __str__(self):
        return f"Animal[{self.name}, {id(self)=}]"

    def __del__(self):
        print(f"del {str(self)}")

In [32]:
tiger = Animal("tiger")

In [33]:
print(tiger)

Animal[tiger, id(self)=4753221216]


In [34]:
del tiger

del Animal[tiger, id(self)=4753221216]


In [36]:
tiger = Animal("tiger")
hog = Animal("hog", tiger)

print(f"1111: {sys.getrefcount(tiger)=}, {sys.getrefcount(hog)=}")

tiger.food = hog

print(f"2222: {sys.getrefcount(tiger)=}, {sys.getrefcount(hog)=}")

del tiger
del hog

1111: sys.getrefcount(tiger)=3, sys.getrefcount(hog)=2
2222: sys.getrefcount(tiger)=3, sys.getrefcount(hog)=3


In [37]:
import gc

In [38]:
gc.collect()

del Animal[tiger, id(self)=4761043024]
del Animal[hog, id(self)=4761043344]
del Animal[tiger, id(self)=4751449904]
del Animal[hog, id(self)=4751445648]


1696

In [41]:
gc.disable()

In [39]:
import ctypes

In [40]:
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]

In [45]:
tiger = Animal("tiger")
hog = Animal("hog", tiger)
tiger.food = hog

id_tiger = id(tiger)
id_hog = id(hog)

print(f"1111: {sys.getrefcount(tiger)=}, {sys.getrefcount(hog)=}, {id_tiger=}, {id_hog=}")

for name, id_ in (("tiger", id_tiger), ("hog", id_hog)):
    print(f"2222: {name}: {PyObject.from_address(id_).refcnt=}")

del tiger
del hog

for name, id_ in (("tiger", id_tiger), ("hog", id_hog)):
    print(f"3333: {name}: {PyObject.from_address(id_).refcnt=}")

gc.collect()

for name, id_ in (("tiger", id_tiger), ("hog", id_hog)):
    print(f"4444: {name}: {PyObject.from_address(id_).refcnt=}")

1111: sys.getrefcount(tiger)=3, sys.getrefcount(hog)=3, id_tiger=4748290128, id_hog=4750224480
2222: tiger: PyObject.from_address(id_).refcnt=2
2222: hog: PyObject.from_address(id_).refcnt=2
3333: tiger: PyObject.from_address(id_).refcnt=1
3333: hog: PyObject.from_address(id_).refcnt=1
del Animal[tiger, id(self)=4748290128]
del Animal[hog, id(self)=4750224480]
4444: tiger: PyObject.from_address(id_).refcnt=-4294967290
4444: hog: PyObject.from_address(id_).refcnt=0


In [46]:
tiger = Animal("tiger")

In [47]:
sys.getsizeof(tiger)

48

In [48]:
tiger.__dict__

{'name': 'tiger', 'food': None}

In [49]:
sys.getsizeof(tiger.__dict__)

208

In [51]:
sys.getsizeof({}), sys.getsizeof({1: 11})

(64, 224)

In [52]:
dct = {}  # PyObject[id=123]

for i in range(8):
    dct[i] = i
    print(f"{i=}, {len(dct)=}, {sys.getsizeof(dct)}")

i=0, len(dct)=1, 224
i=1, len(dct)=2, 224
i=2, len(dct)=3, 224
i=3, len(dct)=4, 224
i=4, len(dct)=5, 224
i=5, len(dct)=6, 352
i=6, len(dct)=7, 352
i=7, len(dct)=8, 352


In [53]:
import weakref

In [54]:
tiger = Animal("tiger")
print(f"1111: {sys.getrefcount(tiger)=}")

weak = weakref.ref(tiger)
print(weak)
print(f"2222: {sys.getrefcount(tiger)=}")

weakref.finalize(tiger, lambda: print("tiger finished"))
print(f"3333: {sys.getrefcount(tiger)=}")


obj = weak()
print(f"4444: {sys.getrefcount(tiger)=}")


del tiger, obj

print("===========")


obj = weak()
print(f"{obj=}")
print(f"5555: {sys.getrefcount(obj)=}")

1111: sys.getrefcount(tiger)=2
<weakref at 0x11c0816c0; to 'Animal' at 0x11b52a270>
2222: sys.getrefcount(tiger)=2
3333: sys.getrefcount(tiger)=2
4444: sys.getrefcount(tiger)=3
del Animal[tiger, id(self)=4753367664]
tiger finished
obj=None
5555: sys.getrefcount(obj)=3221225480


In [76]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


class PointSlots:
    __slots__ = ("x", "y")

    def __init__(self, x, y):
        self.x = x
        self.y = y


class PointFakeSlots(Point):
    __slots__ = ("x", "y")

    def __init__(self, x, y):
        self.x = x
        self.y = y

In [57]:
p = Point(10, 20)
ps = PointSlots(10, 20)

In [58]:
p.__dict__

{'x': 10, 'y': 20}

In [59]:
ps.__dict__

AttributeError: 'PointSlots' object has no attribute '__dict__'

In [60]:
Point.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 1,
              '__init__': <function __main__.Point.__init__(self, x, y)>,
              '__static_attributes__': ('x', 'y'),
              '__dict__': <attribute '__dict__' of 'Point' objects>,
              '__weakref__': <attribute '__weakref__' of 'Point' objects>,
              '__doc__': None})

In [61]:
PointSlots.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 7,
              '__slots__': ('x', 'y'),
              '__init__': <function __main__.PointSlots.__init__(self, x, y)>,
              '__static_attributes__': ('x', 'y'),
              'x': <member 'x' of 'PointSlots' objects>,
              'y': <member 'y' of 'PointSlots' objects>,
              '__doc__': None})

In [62]:
p.x, p.y, ps.x, ps.y

(10, 20, 10, 20)

In [63]:
p.z = 30

In [64]:
p.z

30

In [65]:
ps.z

AttributeError: 'PointSlots' object has no attribute 'z'

In [66]:
ps.x = 50

In [67]:
ps.x

50

In [68]:
sys.getsizeof(p), sys.getsizeof(ps)

(48, 48)

In [70]:
fake = PointFakeSlots(10, 20)

In [71]:
fake.x, fake.y

(10, 20)

In [72]:
fake.x = 50

In [73]:
fake.x, fake.y

(50, 20)

In [74]:
fake.__dict__

{}

In [75]:
PointFakeSlots.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 16,
              '__slots__': ('x', 'y'),
              '__init__': <function __main__.PointFakeSlots.__init__(self, x, y)>,
              '__static_attributes__': ('x', 'y'),
              'x': <member 'x' of 'PointFakeSlots' objects>,
              'y': <member 'y' of 'PointFakeSlots' objects>,
              '__doc__': None})

In [77]:
fake.z = 99

In [78]:
fake.z

99

In [79]:
fake.__dict__

{'z': 99}

In [80]:
def create_objects(cls, n):
    return [cls(10, 20) for _ in range(n)]


def process_objects(objs):
    for obj in objs:
        obj.x = 1
        obj.y = obj.x
        obj.x = 5
        obj.x = obj.y
        obj.y = 9

In [107]:
%%time

N = 1_000_000

point_objs = create_objects(Point, N)

CPU times: user 74.8 ms, sys: 14 ms, total: 88.8 ms
Wall time: 88 ms


In [108]:
%%time

N = 1_000_000

slots_objs = create_objects(PointSlots, N)

CPU times: user 70.1 ms, sys: 9.37 ms, total: 79.5 ms
Wall time: 78.9 ms


In [124]:
%%timeit -n 100

process_objects(point_objs)

20.7 ms ± 221 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [125]:
%%timeit -n 100

process_objects(slots_objs)

20.3 ms ± 264 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
